## Day 18: Settlers of The North Pole

https://adventofcode.com/2018/day/18

### Part 1

Another case of following the instructions. 

In [1]:
from collections import defaultdict, Counter
from pyrsistent import pmap


def adjacent(xy):
    x, y = xy
    
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if dx != 0 or dy != 0:
                yield (x + dx, y + dy)
                
                
def next_acre(acres, xy):
    c = Counter(acres[n] for n in adjacent(xy) if n in acres)
    
    if acres[xy] == '.' and c['|'] >= 3:
        return '|'
    elif acres[xy] == '|' and c['#'] >= 3:
        return '#'
    elif acres[xy] == '#' and (c['|'] < 1 or c['#'] < 1):
        return '.'
    else:
        return acres[xy]
    

def minute(acres):
    return pmap({xy: next_acre(acres, xy) for xy in acres})


def n_minutes(acres, n):
    for _ in range(n):
        acres = minute(acres)
        
    return acres


def parse_acres(acres_data):
    acres = pmap()
    
    for y, line in enumerate(acres_data):
        for x, c in enumerate(line.rstrip()):
            acres = acres.set((x, y), c)
            
    return acres


def print_acres(acres):
    xs, ys = zip(*acres)
    
    for y in range(min(ys), max(ys) + 1):
        line = ''
        
        for x in range(min(xs), max(xs) + 1):
            line += acres[(x, y)]
            
        print(line)
            
            
test_acres = parse_acres('''.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.'''.splitlines())

print_acres(test_acres)

.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.


In [2]:
a = test_acres
for n in range(1, 11):
    a = minute(a)
    print(f'After {n} minutes:')
    print_acres(a)

After 1 minutes:
.......##.
......|###
.|..|...#.
..|#||...#
..##||.|#|
...#||||..
||...|||..
|||||.||.|
||||||||||
....||..|.
After 2 minutes:
.......#..
......|#..
.|.|||....
..##|||..#
..###|||#|
...#|||||.
|||||||||.
||||||||||
||||||||||
.|||||||||
After 3 minutes:
.......#..
....|||#..
.|.||||...
..###|||.#
...##|||#|
.||##|||||
||||||||||
||||||||||
||||||||||
||||||||||
After 4 minutes:
.....|.#..
...||||#..
.|.#||||..
..###||||#
...###||#|
|||##|||||
||||||||||
||||||||||
||||||||||
||||||||||
After 5 minutes:
....|||#..
...||||#..
.|.##||||.
..####|||#
.|.###||#|
|||###||||
||||||||||
||||||||||
||||||||||
||||||||||
After 6 minutes:
...||||#..
...||||#..
.|.###|||.
..#.##|||#
|||#.##|#|
|||###||||
||||#|||||
||||||||||
||||||||||
||||||||||
After 7 minutes:
...||||#..
..||#|##..
.|.####||.
||#..##||#
||##.##|#|
|||####|||
|||###||||
||||||||||
||||||||||
||||||||||
After 8 minutes:
..||||##..
..|#####..
|||#####|.
||#...##|#
||##..###|
||##.###||
|||####|||
||||#|||||
||||||

In [3]:
Counter(a.values())

Counter({'#': 31, '.': 32, '|': 37})

In [4]:
Counter(n_minutes(parse_acres(open('input', 'r')), 10).values())

Counter({'#': 510, '.': 894, '|': 1096})

In [5]:
1096*510

558960

### Part 2

Try to find a loop. This could take a while as it's a bit slow. 

In [10]:
%time print_acres(n_minutes(parse_acres(open('input', 'r')), 100))

...........................................##||||.
...........................................####|||
.............................................###||
..............................................##||
........................|||....................##|
......................|||||||..................##|
....................|||||##||||.................##
....................|||######||||...............##
...................|||###..####|||................
...................||##......####|................
..................|||##........###................
...................||##...........................
...................|||............................
..|...............................................
||||..............................................
|||#..............................................
||###.........|...................................
|##..........|||..................................
||##........##|||.................................
||##.........##||......||......

In [14]:
def find_loop(acres):
    acres_seen = {acres: 0}
    for n in range(1, 100000):
        if n % 100 == 0:
            print(f'After {n} minutes:')
            print_acres(acres)
        acres = minute(acres)
        if acres in acres_seen:
            return acres_seen[acres], n
        acres_seen[acres] = n
            
acres = parse_acres(open('input', 'r'))
find_loop(acres)

After 100 minutes:
..........................................##|||||.
..........................................####|||.
............................................###|||
..............................................##||
.........................|....................##||
.......................|||||...................##|
.....................|||||||||.................##|
....................||||####||||................##
....................||########||||..............##
...................|||##....####||................
...................||##.......####................
...................|||#.........##................
....................||............................
..................................................
.|||..............................................
||||..............................................
||###.............................................
||##.........|||..................................
||##........##||..................................
|||##.......

(402, 430)

Fortunately it's found relatively quickly, the state after 402 minutes is repeated 28 minutes later. I'll put this into a function if it doesn't work.

In [16]:
big_number = 1000000000
(big_number - 402) % 28

10

In [17]:
Counter(n_minutes(acres, 412).values())

Counter({'#': 350, '.': 1556, '|': 594})

In [18]:
350*594

207900

Jolly good, it works.